<a href="https://colab.research.google.com/github/vincelemke99/Bachelor-Thesis/blob/main/lead_predication_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mlens

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

# Import machine learning libraries
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    roc_curve,
    confusion_matrix
)
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.impute import SimpleImputer


In [ ]:
from google.colab import auth
auth.authenticate_user()
print("Authenticated")

In [ ]:
PROJECT_ID = 'prj-snd-ew3-vschmitt'

In [ ]:
from google.cloud import bigquery
client = bigquery.Client(project=PROJECT_ID)

In [ ]:
BQ_PROJECT = PROJECT_ID
BQ_DATASET = 'lead_prediction'
BQ_TABLE = 'trainings_data_converted'

In [ ]:
# SQL query to select data
query = f"""
SELECT *
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`
"""

try:
    # Execute the query and convert the result to a pandas DataFrame
    df = client.query(query=query).to_dataframe()
    print("Query executed successfully and DataFrame is ready.")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
df.shape

In [ ]:
# Identify duplicate lead_ids
duplicate_lead_ids = df[df.duplicated('lead_id', keep=False)]['lead_id'].unique()

# Filter rows to drop
rows_to_drop = df[(df['lead_id'].isin(duplicate_lead_ids)) & (df['is_converted'] == False)]

# Drop rows
# Keep only unique lead_ids (this is optional if the above steps suffice)
df = df.drop_duplicates(subset='lead_id')

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.drop(['lead_id',], axis="columns", inplace=True)

In [ ]:
df.rename(columns={'name': 'Name_Describe','geschlecht' : 'Gender', 'produkt_zeitraum_c' : 'Study_Model',
       'produkt_art_der_ausbildung_c' : 'Type_of_Study_Program', 'produkt_standort' : 'Study_Location',
       'produkt_fachbereich' : 'Faculty', 'produkt_name': 'Study_Program','product_interest_type' : 'Conversion_Type',
       'lead_quelle_detail' : 'Lead_Source_Detail', 'domain': 'Domain', 'has_phone' : 'has_Phone', 'lead_source' : 'Lead_Source', 'Pagepath', 'time_OnSite' : 'Time_on_Site' , 'bounce' : 'Bounce' },inplace=True)
df.head()

In [ ]:
df.isna().sum().to_frame(name='Missing Values').sort_values('Missing Values', ascending=False)

In [ ]:
# Calculate the percentage of missing values for each column
percentage_missing = df.isnull().sum() * 100 / len(df)
# Create a DataFrame to display the missing values
missing_values_df = percentage_missing.to_frame(name='Missing Values').sort_values('Missing Values', ascending=False)
# Display the DataFrame
missing_values_df

In [ ]:
df['Study_Program'].value_counts()

In [ ]:
df['Study_Program'].value_counts()

In [ ]:
df['Semester'] = df['Semester'].fillna('Wintersemester')
df['Study_Program'] = df['Study_Program'].fillna('Psychologie, B.Sc.')
df['Lead_Source_Detail'] = df['Lead_Source_Detail'].fillna('Google Ads')
df['Domain'] = df['Domain'].fillna('gmail.com')
df['Gender'] = df['gender'].fillna('weiblich')
df['Conversion_Type'] = df['Conversion_Type'].fillna('Infomaterial')
df['Pagepath'] = df['Pagepath'].fillna('https://www.hs-fresenius.de/infomaterial/')
df['Operating_System'] = df['Operating_System'].fillna('Windows 10')
df['Browser'] = df['Browser'].fillna('Safari')
df['Language'] = df['Language'].fillna('de-de')
df['Device_Category'] = df['Device_Category'].fillna('mobile')
df['Bounce'] = df['Bounce'].fillna('true')
df['ManualAdContent'] = df['ManualAdContent'].fillna('ads')
df['ManualTerm'] = df['ManualTerm'].fillna('responsive search ads')

In [ ]:
df.describe()

In [ ]:
df['Time_on_Site'] = df['Time_on_Site'].fillna(40.0)

In [ ]:
df.isna().sum().to_frame(name='Missing Values').sort_values('Missing Values', ascending=False)

In [ ]:
plt.figure(figsize=(5,5))
sns.countplot(df['is_converted']);
plt.title('No of Unsuccessful Leads vs Succesful Converted Leads', color='blue')
plt.xthicks(np.arange(2),('Unscuccesful','Successful'))

In [ ]:
def plot_frequency_and_percentage(feature, leads_df, category_df, another_row=False, height=8, ylabels=[]):
    if another_row:
        fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(10, height * 2))
    else:
        fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(15, 6))
    fig.subplots_adjust(hspace=0.3)
    fig.subplots_adjust(wspace=0.3)

    ## plot the frequency plot for each category in the required column
    ax1.set_title('Frequency Plot of {0}'.format(feature), color='blue')
    ax1.set_ylabel(feature)
    ax1.set_xlabel('count')
    sns.countplot(y=feature, data=category_df, order=category_df[feature].value_counts().index, ax=ax1, color='green')
    if len(ylabels) > 0:
        ax1.set_yticklabels(ylabels)

    ## plot the value percentage in each sub-category wrt the label
    ax2.set_title('Lead Converted label %', color='blue')
    ax2.set_ylabel(feature)
    ax2.set_xlabel('percentage')
    leads_df.iloc[1].sort_values().plot(kind='barh', ax=ax2, color='orange')
    if len(ylabels) > 0:
        ax2.set_yticklabels(ylabels)


In [ ]:
df['Lead_Source'].value_counts()

In [ ]:
plot_frequency_and_percentage('Lead_Source', crosstab_df, leadscore_df.sort_values(by='Lead_Source', ascending=False), True, 6)

In [ ]:
plot_frequency_and_percentage('Lead_Source_Detail', crosstab_df, leadscore_df.sort_values(by='Lead_Source_Detail', ascending=False), True, 6)

In [ ]:
plot_frequency_and_percentage('Gender', crosstab_df, leadscore_df.sort_values(by='Gender', ascending=False), True, 6)

In [ ]:
plot_frequency_and_percentage('Study_Program', crosstab_df, leadscore_df.sort_values(by='Study_Program', ascending=False), True, 6)

In [ ]:
plot_frequency_and_percentage('Study_Model', crosstab_df, leadscore_df.sort_values(by='Study_Model', ascending=False), True, 6)